# OpenAI Function Calling In LangChain

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、Pydantic语法](#二、Pydantic语法)
     - [2.1 简单创建Python类](#2.1-简单创建Python类)
     - [2.2 使用Pydantic创建类](#2.2-使用Pydantic创建类)
 - [三、 基于Pydantic的OpenAI函数定义](#三、基于Pydantic的OpenAI函数定义)
     - [3.1 构造OpenAI的function](#3.1-构造OpenAI的function)
     - [3.2 通过langchain使用function](#3.2-通过langchain使用function)
     - [3.3 强制使用function](#3.3-强制使用function)
 - [四、使用function](#四、使用function)
     - [4.1 链式使用](#3.1-链式使用)
     - [4.2 使用多个function](#4.2-使用多个function)

# 一、设置OpenAI-API-Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [ ]:
# 下载需要的包python-dotenv和openai
# 如果你需要查看安装过程日志，可删除 -q
!pip install -q python-dotenv
!pip install -q openai

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [2]:
"""
1.find_dotenv()寻找并定位.env文件的路径
2.load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
3.如果你设置的是全局的环境变量，这行代码则没有任何作用
"""
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.environ['OPENAI_API_KEY']

# 二、Pydantic语法

Pydantic是python的“数据验证库”。
- 与python类型注释一起工作。但是，与静态类型检查不同，它们在运行时被积极地用于数据验证和转换。
- 提供内置方法来序列化/反序列化模型到JSON，字典等。
- LangChain利用Pydantic创建JsONScheme描述函数。

在标准python中，你可以这样创建一个类:

## 2.1 简单创建Python类

In [3]:
from typing import List
from pydantic import BaseModel, Field

In [4]:
# 创建User类
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [5]:
# 生成user对象
foo = User(name="Joe",age=32, email="joe@gmail.com")

# 输出foo中 name字段
foo.name

'Joe'

In [6]:
# 生成user对象
foo = User(name="Joe",age="bar", email="joe@gmail.com")

# 输出foo中 name字段
# name字段填写的是字符串类型，但能够创建成功并输出
foo.age

'bar'

## 2.2 使用Pydantic创建类

In [7]:
# 使用 Pydantic创建pUser类别，说明age使用int类型
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [8]:
# 生成user对象
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

# 输出foo中 name字段
foo_p.name

'Jane'

In [9]:
# 使用了pydantic，由于age不是int，因此会报错。输出报错内容
try:
    foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")
except Exception as e:
    print(e)

1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)


In [10]:
class Class(BaseModel):
    students: List[pUser]

In [11]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

# 三、基于Pydantic的OpenAI函数定义

## 3.1 构造OpenAI的function

In [12]:
# 定义WeatherSearch
# WeatherSearch的function需要写注释完成函数的description
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [13]:
# 导入langchain
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [14]:
# 转化为openai function calling模式
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [15]:
# WeatherSearch1没有写注释，会报错
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")
  

In [16]:
# 使用了pydantic，由于age不是int，因此会报错。输出报错内容
convert_pydantic_to_openai_function(WeatherSearch1)

In [18]:
# 构造WeatherSearch2，不对参数注释
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [19]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

## 3.2 通过langchain使用function

In [20]:
# 导入ChatOpenAI
from langchain.chat_models import ChatOpenAI

In [22]:
model = ChatOpenAI()

In [23]:
# 使用openai function功能
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [24]:
# 直接在构造模型中声明functions，对话时不需要在声明
model_with_function = model.bind(functions=[weather_function])
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

## 3.3 强制使用function

In [25]:
# 强制使用function，在模型构建时声明function_call
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [26]:
# 基于已经声明的function对话，能够调用function
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [27]:
# 强制使用function的模型，即使prompt与函数description无关也会命中
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [28]:
# 未强制使用function的模型，prompt与函数description无关不会命中
model_with_function.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)

# 四、使用function

## 4.1 链式使用

In [29]:
# 引入需要的环境
from langchain.prompts import ChatPromptTemplate

In [30]:
# 使用预定义模版创建聊天代码
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [31]:
# 使用prompt + model_with_function 组合
chain = prompt | model_with_function

In [32]:
# 创建命中function的问答
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [33]:
# 创建未命中function的问答
chain.invoke({"input": "hi"})

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)

## 4.2 使用多个function

我们可以传递一组函数，让LLM根据问题上下文决定使用哪个函数。

In [34]:
# 创建 ArtistSearch function
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [35]:
# 组装WeatherSearch和ArtistSearch 函数
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [36]:
# 将两个function放入模型
model_with_functions = model.bind(functions=functions)

In [37]:
# 命中 WeatherSearch function
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [38]:
# 命中 ArtistSearch function
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{\n  "artist_name": "taylor swift",\n  "n": 3\n}'}}, example=False)

In [39]:
# 命中都未命中
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)